# مصنف الصور البسيط

يُظهر هذا الدفتر كيفية تصنيف الصور باستخدام شبكة عصبية مدربة مسبقًا.

**ما ستتعلمه:**
- كيفية تحميل واستخدام نموذج مدرب مسبقًا
- معالجة الصور مسبقًا
- إجراء التنبؤات على الصور
- فهم درجات الثقة

**حالة الاستخدام:** التعرف على الأشياء في الصور (مثل "قط"، "كلب"، "سيارة"، إلخ)

---


## الخطوة 1: استيراد المكتبات المطلوبة

لنقم باستيراد الأدوات التي نحتاجها. لا تقلق إذا لم تفهم جميع هذه الأدوات بعد!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## الخطوة 2: تحميل النموذج المدرب مسبقًا

سنستخدم **MobileNetV2**، وهو شبكة عصبية تم تدريبها مسبقًا على ملايين الصور.

هذا ما يُعرف بـ **التعلم بالنقل** - استخدام نموذج قام شخص آخر بتدريبه!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## الخطوة 3: وظائف المساعدة

لنقم بإنشاء وظائف لتحميل الصور وتحضيرها لنموذجنا.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## الخطوة 4: اختبار على صور نموذجية

لنحاول تصنيف بعض الصور من الإنترنت!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### صنّف كل صورة


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## الخطوة 5: جرب صورك الخاصة!

استبدل الرابط أدناه بأي رابط صورة تريد تصنيفها.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 ماذا حدث للتو؟

1. **قمنا بتحميل نموذج مُدرب مسبقًا** - تم تدريب MobileNetV2 على ملايين الصور  
2. **قمنا بمعالجة الصور مسبقًا** - تم تغيير حجمها وتنسيقها لتناسب النموذج  
3. **النموذج قام بالتنبؤات** - أعطى احتمالات لـ 1000 فئة من الكائنات  
4. **قمنا بفك تشفير النتائج** - تم تحويل الأرقام إلى تسميات مفهومة للبشر  

### فهم درجات الثقة

- **90-100%**: ثقة عالية جدًا (تقريبًا صحيحة بالتأكيد)  
- **70-90%**: ثقة عالية (على الأرجح صحيحة)  
- **50-70%**: ثقة متوسطة (قد تكون صحيحة)  
- **أقل من 50%**: ثقة منخفضة (غير مؤكدة)  

### لماذا قد تكون التنبؤات خاطئة؟

- **زاوية أو إضاءة غير مألوفة** - النموذج تم تدريبه على صور تقليدية  
- **وجود عدة كائنات** - النموذج يتوقع وجود كائن رئيسي واحد  
- **كائنات نادرة** - النموذج يعرف فقط 1000 فئة  
- **صورة منخفضة الجودة** - الصور الضبابية أو ذات البيكسلات المنخفضة أصعب في التنبؤ  


## 🚀 الخطوات التالية

1. **جرب صورًا مختلفة:**
   - ابحث عن صور على [Unsplash](https://unsplash.com)
   - انقر بزر الماوس الأيمن → "نسخ عنوان الصورة" للحصول على الرابط

2. **جرب:**
   - ماذا يحدث مع الفن التجريدي؟
   - هل يمكنه التعرف على الأشياء من زوايا مختلفة؟
   - كيف يتعامل مع وجود عدة أشياء في الصورة؟

3. **تعلم المزيد:**
   - استكشف [دروس الرؤية الحاسوبية](../lessons/4-ComputerVision/README.md)
   - تعلم كيفية تدريب مصنف الصور الخاص بك
   - افهم كيفية عمل الشبكات العصبية الالتفافية (CNNs)

---

## 🎉 تهانينا!

لقد قمت ببناء مصنف صور باستخدام شبكة عصبية متقدمة!

هذه التقنية نفسها تُستخدم في:
- Google Photos (تنظيم صورك)
- السيارات ذاتية القيادة (التعرف على الأشياء)
- التشخيص الطبي (تحليل الأشعة السينية)
- مراقبة الجودة (اكتشاف العيوب)

استمر في الاستكشاف والتعلم! 🚀



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
